In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

train = pd.read_csv('~/Downloads/train.csv')
test = pd.read_csv('~/Downloads/test.csv')
y_train = train['Label']

print(train.shape)
print(test.shape)

(1599, 41)
(400, 40)


In [57]:
#na_cols = train.columns[train.isna().any()].tolist()
#na_cols = test.columns[test.isna().any()].tolist()
# Fill missing scores using mean
score_values = train[['I1','I2','I3','I4','I5','I6','I7','I8','I9','I10','I11','I12','I13']].mean()
train = train.fillna(score_values)
test = test.fillna(score_values)
# Fill missing catagorical data using 'NotSpecific'
cols = ['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12', 'C13','C14','C15','C16','C17','C18','C19','C20','C21','C22','C23','C24','C25','C26']
train = train.fillna({k: 'NotSpecified' for k in cols})
test = test.fillna({k: 'NotSpecified' for k in cols})

In [58]:
def set_unique_categories(category_columns, train, test):
    """
    Set unique categories (include both in training and testing data) for each column so we can do one hot encoding
    @Param category_columns: column name
    @Param train: training data
    @Param test: testing data
    """
    for column in category_columns:
        unique_elements = pd.concat([train[column],test[column]]).unique().tolist()
        train[column] = train[column].astype('category').cat.set_categories(unique_elements)
        test[column] = test[column].astype('category').cat.set_categories(unique_elements)
        
# One hot encoding 
category_columns = {'C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12', 'C13','C14','C15','C16','C17','C18','C19','C20','C21','C22','C23','C24','C25','C26'}
set_unique_categories(category_columns, train, test)
one_hot_train = pd.get_dummies(train)
one_hot_test = pd.get_dummies(test)

x_train = one_hot_train.drop(['Id','Label'], axis=1)
x_test = one_hot_test.drop(['Id'], axis=1)

print(x_train.shape)
print(x_test.shape)
x_test.head(1)                            



(1599, 13104)
(400, 13104)


,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,...,C26_ed9e6b03,C26_68d9ada1,C26_9a333cac,C26_abc00283,C26_8fa55041,C26_a6dec5b6,C26_491eeeef,C26_8f9d38b3,C26_80dd0a5b,C26_7132fed8
0,3.515487,-1,35.562802,7.921739,8020.0,26.0,6.0,0.0,80.0,0.55531,...,0,0,0,0,0,0,0,0,0,0


In [59]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

/Users/mingxie/.pyenv/versions/3.7.0/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/mingxie/.pyenv/versions/3.7.0/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/mingxie/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [62]:
#clf = LogisticRegression(random_state=0, C=0.25, class_weight={0: 1, 1: 10}).fit(x_train, y_train)
#clf = LogisticRegression().fit(x_train, y_train)
#clf = SGDClassifier().fit(x_train, y_train)
#clf = LinearSVC().fit(x_train, y_train)
#clf = DecisionTreeClassifier().fit(x_train, y_train)
#clf = GradientBoostingClassifier().fit(x_train, y_train)
clf = RandomForestClassifier().fit(x_train, y_train)
pred = clf.predict(x_test)

/Users/mingxie/.pyenv/versions/3.7.0/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [63]:
submission = pd.read_csv('~/Downloads/submission.csv')
submission['Label'] = pred
submission.to_csv('~/Downloads/forest_submission.csv', index=False)